In [1]:
from Utils.FS import file
from Utils.tensorflow_helper import show_graph
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from scipy.sparse import coo_matrix, dok_matrix
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
import tensorflow as tf
import math
import TextPreprocess.words2dict as words2dict
from tensorflow.python.layers import core as layers_core
from tensorflow.python.client import timeline
import time
from DataLoader import GloVe
from TextPreprocess.sequences import Sequences
from TextPreprocess.Tokenizer.RegExp import tokenize
import Utils.pandas_helper as ph

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
np.random.seed(1234)
WORD_DIM = 300
WORD_COUNT = 400000+3

In [4]:
glove = GloVe.load2('./data/GloVe/glove.6B.{}d.txt'.format(WORD_DIM))

Start: Loading Glove Model
End: Loaded 400000 rows.


In [5]:
# emb: Symbol to float32 of fixed DIMENSION
# Create an index mapping, index to symbol, symbol to index

class Embedding:
    def __init__(self, emb, verbose = False):
        # assert emb is dictionary and each entry has same dimension
        self.emb = emb
        self.dim = len(self.emb[list(self.emb.keys())[0]])
        self.emb['<UNK>'] = [0. for i in range(self.dim)]
        self.emb['<PAD>'] = [1. for i in range(self.dim)]
        self.emb['<GO>'] = [-1. for i in range(self.dim)]
        
        self.build_dicts()
        
        if verbose:
            self.describe()
        
    def describe(self):
        print('Embedding Dimension: {}'.format(self.dim))
        print('Embedding Symbols: {}'.format(len(self.emb)))
        print('Index to symbol: {}'.format([(i, self.idx2Sym[i]) for i in range(10)]))
        
    def getIndex(self, symbol):
        if symbol in self.sym2Idx:
            return self.sym2Idx[symbol]
        else:
            return self.sym2Idx['<UNK>']

    def getEmb(self, symbol):
        return self.emb[self.idx2Sym[self.getIndex(symbol)]]
    
    def getSymbols(self, indices):
        return [self.idx2Sym[idx] for idx in indices]

    def getNumpyArray(self):
        return np.array([self.emb[self.idx2Sym[idx]] for idx in range(len(self.emb))])
    
    def build_dicts(self):
        self.sym2Idx = {}
        index = 0
        for key in self.emb.keys():
            self.sym2Idx[key] = index
            index += 1
            
        self.idx2Sym = { v:k for k, v in self.sym2Idx.items()}

glove_emb = Embedding(glove, verbose=True)

Embedding Dimension: 300
Embedding Symbols: 400003
Index to symbol: [(0, 'radstadt'), (1, 'hovensa'), (2, 'five-hit'), (3, 'barath'), (4, "d'action"), (5, 'chronically'), (6, 'tragicomedies'), (7, 'keyshia'), (8, 'klimov'), (9, 'kirs')]


In [6]:
df = file.read('data/Quora/train.csv')

from sklearn.model_selection import train_test_split

df.question1 = df.question1.astype(str)
df.question2 = df.question2.astype(str)
df.is_duplicate = df.is_duplicate.astype(float)

df = df.as_matrix(['question1', 'question2', 'is_duplicate'])

data = {}
data['train'], data['test'] = train_test_split(df, test_size = 0.1)

In [7]:
def preprocessQuestion(string):
    try:
        return [glove_emb.getIndex(token.lower()) for token in tokenize(string)]
    except:
        print(string)


def preprocessData(data):
    return [[preprocessQuestion(rec[0]), preprocessQuestion(rec[1]), float(rec[2])] for rec in data]

In [8]:
for i in ['train', 'test']:
    data[i] = preprocessData(data[i])

In [9]:
# Turns iteratable of symbols into padded batch
from functools import lru_cache

class Batcher:
    def __init__(self, sequences, verbose = False):
        self.seqs = sequences
        self.verbose = verbose
        self.size = len(self.seqs)
        self.seq_lens = [len(seq) for seq in self.seqs]
        
        if self.verbose:
            self.describe()
    
    @lru_cache(maxsize=None)
    def max_length(self):
        return max(self.seq_lens)
    
    @lru_cache(maxsize=None)
    def longgest_sequence(self):
        for seq in self.seqs:
            if len(seq) == self.max_length():
                return seq
    
    def describe(self):
        print('Size: {}'.format(self.size))
        print("Longest sequence length: {}".format(self.max_length()))
        bin_width = max(1, self.max_length() // 30)
        plt.hist(self.seq_lens, range(0, self.max_length() + bin_width, bin_width))
        plt.title('Sequence length distribution')
        plt.show()
        
    def batchPadding(self, batch, padding_symbol):
        size = max([len(record) for record in batch])
        result = np.full((len(batch), size), padding_symbol)
        for i in range(len(batch)):
            result[i][:len(batch[i])] = batch[i]
        return result

    def batchMask(self, batch):
        size = max([len(record) for record in batch])
        result = np.full((len(batch), size), 0.0)
        for i in range(len(batch)):
            result[i][:len(batch[i])] = 1.0
        return result
        
    # Same length within the batch, stuffed with padding symbol
    def generator(self, padding_symbol, batch_size=None, epouch=-1):
        if batch_size == None:
            batch_size = self.size
        train = []
        length = []
        while(epouch < 0 or epouch > 0):
            for seq in self.seqs:
                train.append([sym for sym in seq])
                length.append(len(seq))
                if(len(train) == batch_size):
                    yield self.batchPadding(train, padding_symbol), length, self.batchMask(train)
                    train = []
                    length = []
            epouch -= 1
            if self.verbose:
                print('epouch done...')
                
class Batcher2:
    def __init__(self, sequences, verbose = False):
        self.seqs = sequences
        self.size = len(self.seqs)

    def generator(self, batch_size=32, epouch=-1):
        if batch_size == None:
            batch_size = self.size
        train = []
        while(epouch < 0 or epouch > 0):
            for sym in self.seqs:
                train.append([sym])
                if(len(train) == batch_size):
                    yield train
                    train = []
            epouch -= 1
            print('epouch done...')
            
            
# Turn data into batch, where data is iterable over records, record is iterable over fields
class Batcher3:
    def __init__(self, data):
        #assert it is doubly iterable
        self.data = data
        self.size = len(data)
        
    def generator(self, batch_size = 32, epouch = -1):
        batch = []
        while(epouch < 0 or epouch > 0):
            for record in self.data:
                batch.append(record)
                if(len(batch) == batch_size):
                    yield batch
                    batch = []
            epouch -= 1
            print('epouch done...')

In [10]:
batcher = {}
for i in ['train', 'test']:
    batcher[i] = Batcher3(data[i])

In [11]:
LV1_DIM = 10
LV2_STEP = 1
LV2_DIM = 150

DROPOUT_RNN = 0.1
DROP_DENSE = 0.1

In [56]:
def embeddings_initializer(shape):
    with tf.variable_scope("Embeddings_Initializer"):
        in_emb = tf.placeholder(
            dtype = tf.float32, 
            shape = shape, 
            name = "Placeholder"
        )
        
        emb = tf.Variable(
            tf.constant(0.0, shape = shape), 
            trainable=False, 
            name = 'Embeddings', 
            dtype=tf.float32
        )
        
        init_emb = emb.assign(in_emb)
    return in_emb, init_emb, emb

def cells_initializer(num_units, reuse):
    with tf.variable_scope("Cells_Initializer"):
        cells = tf.contrib.rnn.GRUCell(
            num_units = num_units,
            input_size = None,
            activation = tf.tanh,
            reuse = reuse
        )
    return cells


#IN (batch, time, 1)
def simple_embedding(inputs, embeddings):
    with tf.variable_scope("Simple_Embedding"):
        lookup = tf.nn.embedding_lookup(
            params = embeddings,
            ids = inputs,
            partition_strategy='mod',
            name='Embedding_Lookup',
            validate_indices=True,
            max_norm=None
        )

    return lookup

#OUT: (batch, time, dim) float32

#IN (batch, time, dim)
def simple_dynamic_rnn(cell, inputs, lengths):
    with tf.variable_scope("Simple_Dynamic_RNN"):        
        
        batch_size = tf.shape(inputs)[0]
        step_size = tf.shape(inputs)[1]

        outputs, states = tf.nn.dynamic_rnn(
            cell, 
            inputs, 
            dtype = tf.float32, 
            sequence_length = lengths,
            initial_state = cell.zero_state(batch_size, dtype=tf.float32)
        )

        indices = tf.range(0, batch_size) * step_size + (lengths - 1)
        gather = tf.reshape(
            tf.gather(
                tf.reshape(outputs, [-1, cell.output_size]), indices
            ), 
            [-1, cell.output_size]
         )
        
    return gather, outputs
#OUT (batch, dim)

#IN (batch, time, dim)
def simple_encoder(inputs, input_lengths, embeddings, dropout = 0.0, reuse = None):
    with tf.variable_scope('Simple_Encoder'):
        
        emb = simple_embedding(inputs, embeddings)
        
        cell = tf.contrib.rnn.GRUCell(
            num_units = LV2_DIM,
            input_size = None,
            activation = tf.tanh,
            reuse = reuse
        )
        
        cell = tf.contrib.rnn.DropoutWrapper(
            cell,
            input_keep_prob = 1. - dropout,
            output_keep_prob = 1. - dropout,
            state_keep_prob = 1. - dropout,
            variational_recurrent=False,
            input_size=None,
            dtype=None,
            seed=None
        )

        rnn, debug_output = simple_dynamic_rnn(
            cell = cell,
            inputs = emb,
            lengths = input_lengths
        )
        
    return rnn, emb, debug_output

In [57]:
graph = tf.Graph()
with graph.as_default():
    
    with tf.variable_scope("Inputs_Layer"):
    
        #IN
        in_word_indices = [tf.placeholder(tf.int32, (None, None), name = "Q{}_Word_Indices".format(i+1)) for i in range(2)]
        #OUT: (batch, time) int32
        
        #batch_size = [tf.shape(inputs[i], name= "Q{}_Batch_Size".format(i+1))[0] for i in range(2)]
        #steps = [tf.shape(inputs[i], name= "Q{}_Steps".format(i+1))[1] for i in range(2)]
        
        #IN
        in_lengths = [tf.placeholder(tf.int32, (None), name = "Q{}_Lengths".format(i+1)) for i in range(2)]
        #OUT: (batch) int32
        
        #in_word_indices = tf.placeholder(tf.int32, (None, None), name = "Q_Word_Indices")
        
        #in_lengths = tf.placeholder(tf.int32, (None), name = "Q_Lengths")
        
        in_truth = tf.placeholder(tf.float32, (None, 1), name = "Truth")

    with tf.variable_scope("Embeddings_Layer"):
        in_emb, init_emb, emb = embeddings_initializer((WORD_COUNT, WORD_DIM))
        
    with tf.variable_scope("Encoder_Layer"):
        enc_0, debug_emb, debug_output = simple_encoder(in_word_indices[0], in_lengths[0], emb, dropout=DROPOUT_RNN)
        enc_1, _, _ = simple_encoder(in_word_indices[1], in_lengths[1], emb, dropout=DROPOUT_RNN, reuse=True)

    #IN: (batch, dim) x2
    with tf.variable_scope("Prediction_Layer"):
        #pred_in = [tf.placeholder(tf.float32, (None, LV2_DIM)) for i in range(2)]
        #pred_input = tf.concat([pred_in[0], pred_in[1]], 1)
        pred_input = tf.concat([enc_0, enc_1], 1)
        
        pred_weights = tf.Variable(tf.random_uniform([LV2_DIM * 2, 150], -1, 1), name='weights')
        pred_bias = tf.Variable(tf.constant(0.0, shape=[1]), name="bias")
        
        pred_weights_2 = tf.Variable(tf.random_uniform([150, 1], -1, 1), name='weights_2')
        pred_bias_2 = tf.Variable(tf.constant(0.0, shape=[1]), name="bias_2")
        
        pred_1 = tf.nn.tanh(pred_input @ pred_weights + pred_bias)
        
        pred = tf.nn.sigmoid(pred_1 @ pred_weights_2 + pred_bias_2)


    loss = tf.reduce_mean(tf.contrib.keras.losses.binary_crossentropy(in_truth, pred))
    acc = tf.reduce_mean(tf.contrib.keras.metrics.binary_accuracy(in_truth, pred))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    saver = tf.train.Saver()
    tvars = tf.trainable_variables()

/home/hoiy927/project/tf_playground/tf_playground/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [55]:
show_graph(graph.as_graph_def())



In [50]:
with tf.Session(graph = graph) as sess:
    sess.run(tf.global_variables_initializer())
    tvars_vals = sess.run(tvars)

    for var, val in zip(tvars, tvars_vals):
        print(var.name, val)  # Prints the name of the variable alongside its value.

Encoder_Layer/Simple_Encoder/Simple_Dynamic_RNN/rnn/gru_cell/gates/weights:0 [[ 0.03026615 -0.01706933  0.02820878 ...,  0.05793758 -0.07652447
   0.01126526]
 [ 0.00832469 -0.07270081 -0.0095413  ...,  0.03583521  0.00998072
   0.04100773]
 [ 0.00313656 -0.01917002 -0.05016559 ..., -0.00066482 -0.0323074
   0.05582645]
 ..., 
 [ 0.05532869  0.04368872  0.01664559 ..., -0.0618533  -0.01999456
   0.08511061]
 [-0.05780245  0.03946281 -0.01556344 ...,  0.03653974 -0.03240189
   0.08326026]
 [ 0.00733077  0.07540397 -0.05367482 ...,  0.05884577 -0.0142995
  -0.00287525]]
Encoder_Layer/Simple_Encoder/Simple_Dynamic_RNN/rnn/gru_cell/gates/biases:0 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 

In [51]:
num_steps = 2000000
MODEL = './model/q.ckpt'

In [52]:
BATCH_SIZE = 128

def batchPadding(batch, padding_symbol):
    size = max([len(record) for record in batch])
    result = np.full((len(batch), size), padding_symbol)
    for i in range(len(batch)):
        result[i][:len(batch[i])] = batch[i]
    return result

def proc_batch(batch):
    q1 = batchPadding([record[0] for record in batch], glove_emb.getIndex('<PAD>'))
    q2 = batchPadding([record[1] for record in batch], glove_emb.getIndex('<PAD>'))
    q1_lens = [len(record[0]) for record in batch]
    q2_lens = [len(record[1]) for record in batch]
    truth = [[record[2]] for record in batch]
    
    return q1, q2, q1_lens, q2_lens, truth

gen = {}
for i in ['train', 'test']:
    gen[i] = batcher[i].generator(batch_size=BATCH_SIZE, epouch = 1)

In [54]:
def run_graph(graph, model_file, batch_size = 32, force_restart=False, epouch = -1):
    
    with tf.Session(graph = graph) as sess:
        try:
            if force_restart:
                sess.run(tf.global_variables_initializer())
                print('Started training...')
            else:
                saver.restore(sess, model_file)
                print('Restored training...')
        except:
            sess.run(tf.global_variables_initializer())
            print('Failed to restore training, restarting training...')

        
        sess.run(init_emb, feed_dict={in_emb: glove_emb.getNumpyArray()})
        
        epouch_start = time.time()
        epouch_count = 0
        while(epouch < 0 or epouch > 0):
            
            batcher = Batcher3(data['train'])
            total = batcher.size
            generator = batcher.generator(batch_size = batch_size, epouch = 1)
            count = 0
            total_loss = 0
            print('Epouch {}...'.format(epouch_count + 1))
            for batch in generator:

                start = time.time()
                train_q1, train_q2, train_q1_lengths, train_q2_lengths, train_label = proc_batch(batch)

                feed_dict = {
                    in_word_indices[0]: train_q1,
                    in_word_indices[1]: train_q2,
                    in_lengths[0]: train_q1_lengths,
                    in_lengths[1]: train_q2_lengths,
                    in_truth: train_label
                }

                _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
                end = time.time()
                count += batch_size
                total_loss += loss_val
                print("Progress: {} / {} ({:2.1%}), Avg. Loss: {:.5f}, Batch Loss: {:.5f}, Time / Batch: {:.2f}s, Time Remains: {:.2f}s".format(
                    count, 
                    total, 
                    count / total, 
                    total_loss / count * batch_size, 
                    loss_val,
                    end - start,
                    (time.time() - epouch_start) / count * total - (time.time() - epouch_start)
                ), end='\x1b[2K\r')
                #print("Progress: {0} / {1}, Loss: {:.2f}, Time / Batch: {:.2f}s, Time Remains: {:.2f}s".format(count, total, loss_val, end - start, (end - start) * (total - count) / batch), end="\r")

            epouch -= 1
            epouch_count += 1
            

run_graph(graph, MODEL, batch_size=2048, force_restart=True)
            
            

Started training...
Epouch 1...


KeyboardInterrupt: 

In [ ]:
DEBUG_SIZE = 1000

with tf.Session(graph = graph) as session:
    try:
        saver.restore(session, MODEL)
        print('Restored training...')
        #session.run(tf.global_variables_initializer())
        #print('Restarting training...')
    except:
        session.run(tf.global_variables_initializer())
        print('Restarting training...')
        

    
    #run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    #run_metadata = tf.RunMetadata()
    
    #tvars_vals = session.run(tvars)
    #for var, val in zip(tvars, tvars_vals):
    #    print(var.name, val)  # Prints the name of the variable alongside its value.

    #for name in session.run( tf.report_uninitialized_variables( tf.global_variables( ) ) ):
    #    print(name)
    
    
    average_loss = 0
    average_max_loss = 0
    start = time.time()
    
    for step in range(num_steps):
        
        train_q1, train_q2, train_q1_lengths, train_q2_lengths, train_label = proc_batch(next(gen['train']))
        
        feed_dict = {
            in_word_indices[0]: train_q1,
            in_word_indices[1]: train_q2,
            in_lengths[0]: train_q1_lengths,
            in_lengths[1]: train_q2_lengths,
            in_truth: train_label
        }
        
        #_, loss_val = session.run([optimizer, loss], feed_dict=feed_dict, options=run_options, run_metadata=run_metadata)
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        
        if step % DEBUG_SIZE == 0:
            if step > 0:
                average_loss /= DEBUG_SIZE
                print('Total time for {0} steps: {1:.2f}s, each step: {2:.2f}s'.format(DEBUG_SIZE, time.time()-start, (time.time()-start) / DEBUG_SIZE))
                print('Average mean loss at step ', step, ': ', average_loss)
                average_loss = 0
                
                avg_loss_val = 0
                avg_acc_val = 0
                
                gen['test'] = Batcher3(data['test']).generator(epouch = 1, batch_size=1024)
                count = 0
                for batch in gen['test']:
                    test_q1, test_q2, test_q1_lengths, test_q2_lengths, test_label = proc_batch(batch)
                    feed_dict = {
                        in_word_indices[0]: test_q1,
                        in_word_indices[1]: test_q2,
                        in_lengths[0]: test_q1_lengths,
                        in_lengths[1]: test_q2_lengths,
                        in_truth: test_label
                    }

                    loss_val, acc_val = session.run([loss, acc], feed_dict=feed_dict)
                    avg_loss_val += loss_val
                    avg_acc_val += acc_val
                    count += 1
                
                print('Testing Set batch loss: {0}, acc {1}:'.format(avg_loss_val/count, avg_acc_val/count))
                
                start = time.time()
                
                
        if step % DEBUG_SIZE == 0:
            save_path = saver.save(session, MODEL)
            print("Model saved in file: %s" % save_path)
            
            # Create the Timeline object, and write it to a json
            #tl = timeline.Timeline(run_metadata.step_stats)
            #ctf = tl.generate_chrome_trace_format()
            #with open('timeline.json', 'w') as f:
            #    f.write(ctf)


In [ ]:
DEBUG_SIZE = 100
with tf.Session(graph=graph) as session:
    saver.restore(session, MODEL)
    print('Restored model...')
    
    average_loss = 0
    start = time.time()
    
    for step in range(num_steps):
        
        test_q1, test_q1_lengths, _ = next(q1_gen['test'])
        test_q2, test_q2_lengths, _ = next(q2_gen['test'])
        test_label = next(label_gen['test'])
        
        feed_dict = {
            inputs[0]: test_q1,
            inputs[1]: test_q2,
            input_lengths[0]: test_q1_lengths,
            input_lengths[1]: test_q2_lengths,
            truth: test_label
        }
        
        loss_val = session.run(loss, feed_dict=feed_dict)
        average_loss += loss_val
        
        if step % DEBUG_SIZE == 0:
            if step > 0:
                average_loss /= DEBUG_SIZE
                print('Total time for {0} steps: {1:.2f}s, each step: {2:.2f}s'.format(DEBUG_SIZE, time.time()-start, (time.time()-start) / DEBUG_SIZE))
                print('Average mean loss at step ', step, ': ', average_loss)
                average_loss = 0
                start = time.time()

In [ ]:
BATCH_SIZE = 2

q1_gen = {}
q2_gen = {}
label_gen = {}
for i in ['train', 'test']:
    q1_gen[i] = q1_batcher[i].generator(glove_emb.getIndex('<PAD>'), batch_size=BATCH_SIZE)
    q2_gen[i] = q2_batcher[i].generator(glove_emb.getIndex('<PAD>'), batch_size=BATCH_SIZE)
    label_gen[i] = label_batcher[i].generator(batch_size=BATCH_SIZE)

with tf.Session(graph=graph) as sess:
    sess.run(init_emb, feed_dict={in_emb: glove_emb.getNumpyArray()})
    
    test_q1, test_q1_lengths, _ = next(q1_gen['test'])
    test_q2, test_q2_lengths, _ = next(q2_gen['test'])
    test_label = next(label_gen['test'])
        
    feed_dict = {
        in_word_indices[0]: test_q1,
        in_word_indices[1]: test_q2,
        in_lengths[0]: test_q1_lengths,
        in_lengths[1]: test_q2_lengths,
        in_truth: test_label
    }
    
    e0_val, e1_val = sess.run([e0, e1], feed_dict=feed_dict)
    print(e0_val)
    print(e1_val)
    

In [ ]:
# debug embedding
WORD_TO_TEST = 'universe'
print(glove[WORD_TO_TEST])
with tf.Session(graph=graph) as sess:
    sess.run(init_emb, feed_dict={in_emb: glove_emb.getNumpyArray()})
    debug_val = sess.run(debug_emb, feed_dict={in_word_indices[0]: [[glove_emb.getIndex(WORD_TO_TEST)]], in_lengths[0]: [1], in_truth: [[1.0]] })
    print(debug_val)

In [72]:
debug_gen = batcher['train'].generator(batch_size = 3)

train_q1, train_q2, train_q1_lengths, train_q2_lengths, train_label = proc_batch(next(debug_gen))

debug_feed_dict = {
    in_word_indices[0]: train_q1,
    in_word_indices[1]: train_q2,
    in_lengths[0]: train_q1_lengths,
    in_lengths[1]: train_q2_lengths,
    in_truth: train_label
}

print(train_q1_lengths)

np.random.seed(1234)

# debug encoder
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(init_emb, feed_dict={in_emb: glove_emb.getNumpyArray()})
    enc_0_val, debug_output_val = sess.run([enc_0, debug_output], feed_dict=debug_feed_dict)
    print(enc_0_val[1])
    print(debug_output_val[1][3])
    print(enc_0_val[2])
    print(debug_output_val[2][8])

[10, 4, 9]
[ 0.14998655 -0.09391348  0.17464606 -0.         -0.00169301 -0.15615971
  0.30296317  0.06012907  0.         -0.05169265 -0.14278217  0.09461938
  0.00987079  0.36617109 -0.02103352 -0.07669148  0.11759835  0.01316576
 -0.         -0.          0.25867811  0.04025744  0.17857444 -0.
 -0.30010751 -0.09162224 -0.00953286 -0.29808503  0.08664629  0.21048747
 -0.13219537 -0.13747475  0.09510694 -0.14797585 -0.01193593 -0.03156843
 -0.00750225  0.30521911  0.16970465  0.14592394  0.04779839  0.25619733
 -0.08198614 -0.11256238 -0.3485522  -0.11879336 -0.09687118  0.02783687
 -0.36816552 -0.11628062 -0.04217453 -0.23391198  0.19854188 -0.04346948
  0.05224289  0.13042878 -0.02688147 -0.21523619  0.15217434 -0.11114306
  0.0862231  -0.03252424 -0.11545172 -0.17975256  0.28948328 -0.11751859
  0.14478113  0.07017203  0.11439773  0.10573865  0.15823565  0.28489
  0.38019195 -0.41692275 -0.26296368 -0.2314489  -0.207469   -0.01810522
 -0.16054942 -0.32907647 -0.19955631  0.2998206   0